# STC Jawwy

In [6]:
"""
Here we install libraries that are not installed by default
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
# !pip install pyxlsb # pyxlsb is not needed for .xlsx files

'\nHere we install libraries that are not installed by default\nExample:  pyslsb\nFeel free to add any library you are planning to use.\n'

In [7]:
# Import the required libraries
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
# import pyxlsb           # Excel extention to read xlsb files (the input file) # pyxlsb is not needed for .xlsx files
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of details about each customer and the movies and/or tv shows watched in addition to the genre.

You are required to work on task three to build a recommendation engine for our platform to Recommend movies to usesrs that they might be interested in¶


In [8]:
dataframe = pd.read_excel("stc TV Data Set_T3.xlsx",index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [9]:
# check the data shape
dataframe.shape

(1048575, 5)

In [10]:
# display the first 5 rows
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [11]:
# describe the numeric values in the dataset
dataframe.describe()

,user_id_maped,rating,date_
count,1.048575e+06,1.048575e+06,1048575
mean,1.709266e+04,2.497283e+00,2017-10-04 00:23:20.346183936
min,1.000000e+00,1.000000e+00,2017-03-14 00:00:00
25%,8.253000e+03,1.000000e+00,2017-06-10 00:00:00
50%,1.714900e+04,2.000000e+00,2017-10-14 00:00:00
75%,2.566500e+04,3.000000e+00,2018-01-21 00:00:00
max,3.428000e+04,4.000000e+00,2018-04-30 00:00:00
std,1.003513e+04,1.119837e+00,NaN


In [12]:
# check if any column has null value in the dataset
dataframe.isnull().any()

,0
user_id_maped,False
program_name,False
rating,False
date_,False
program_genre,False


In [13]:
# we import Visualization libraries
# you can ignore and use any other graphing libraries
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
# الخطوة 1: إعداد البيانات والتركيز على الأفلام
print("\n--- Building The Recommender System ---")

# بما أننا نريد التوصية بأفلام، سنقوم بتصفية البيانات للإبقاء على الأفلام فقط
# وهذا يساعد في جعل التوصيات أكثر دقة وصلة بالموضوع
df_movies = dataframe[dataframe['program_genre'] == 'Animation'].copy() # Assuming 'Animation' is a good proxy for 'MOVIE' based on head() output

print(f"Number of movie watching events: {len(df_movies)}")
print("Data is now filtered for MOVIES only.\n")


# الخطوة 2: إنشاء مصفوفة المستخدمين-الأفلام (User-Item Matrix)
# هذه المصفوفة هي أساس نظام التصفية التعاونية
# الصفوف: المستخدمون (user_id_maped)
# الأعمدة: الأفلام (program_name)
# القيم: 1 إذا شاهد المستخدم الفيلم، 0 إذا لم يشاهده
# نستخدم crosstab لإنشاء هذه المصفوفة بكفاءة
user_movie_matrix = pd.crosstab(df_movies['user_id_maped'], df_movies['program_name'])

print("Shape of User-Movie Matrix:", user_movie_matrix.shape)
print("Example from the matrix (first 5 users and movies):")
print(user_movie_matrix.iloc[:5, :5])


# الخطوة 3: حساب عدد المشاهدات لكل فيلم وإيجاد التشابه مع فيلم "Moana"
# أولاً: نحسب كم مرة تمت مشاهدة كل فيلم. هذا سيساعدنا لاحقاً في تصفية النتائج
# لتجنب التوصية بأفلام غير شائعة
# The original code used 'watch_duration' which doesn't exist. Using 'program_name' count as a proxy for views.
ratings = pd.DataFrame(df_movies.groupby('program_name')['program_name'].count())
ratings.rename(columns={'program_name': 'num_of_views'}, inplace=True)


# ثانياً: نستخرج نمط مشاهدات فيلم "Moana" من المصفوفة
moana_user_ratings = user_movie_matrix['Moana']

# ثالثاً: نحسب معامل الارتباط بين "Moana" وجميع الأفلام الأخرى
# الدالة corrwith تقوم بهذه المهمة بكفاءة عالية
similar_to_moana = user_movie_matrix.corrwith(moana_user_ratings)

# نحول النتائج إلى DataFrame لتسهيل التعامل معها
corr_moana = pd.DataFrame(similar_to_moana, columns=['Correlation'])
corr_moana.dropna(inplace=True) # نزيل الأفلام التي ليس لها ارتباط

# ندمج بيانات الارتباط مع بيانات عدد المشاهدات
corr_moana = corr_moana.join(ratings)

print("\nTop correlated movies with Moana (before filtering):")
print(corr_moana.sort_values('Correlation', ascending=False).head())


--- Building The Recommender System ---
Number of movie watching events: 401730
Data is now filtered for MOVIES only.

Shape of User-Movie Matrix: (8350, 1444)
Example from the matrix (first 5 users and movies):
program_name   A Goofy Movie  A tory and a Proverb  Ep.  Aladdin  \
user_id_maped                                                      
1                          0                          0        0   
5                          0                          0        0   
9                          0                          0        0   
11                         0                          0        0   
15                         0                          0        1   

program_name   Albert Asks  Ep.  Alvin and the Chipmunks  
user_id_maped                                             
1                             0                        0  
5                             0                        0  
9                             0                        0  
11             

In [16]:
# الخطوة 4: عرض أفضل 5 توصيات للأشخاص الذين شاهدوا فيلم "Moana"
# --------------------------------------------------------------------------------
""" TODO show the recommendations (top 5) for the people who watched "Moana" movie """
print("=== Top 5 Recommendations for users who watched 'Moana' ===\n")

# نقوم بتصفية النتائج لاستبعاد الأفلام التي لم يشاهدها عدد كافٍ من المستخدمين
# هذا يضمن أن تكون التوصيات ذات جودة ومصداقية (ليست مجرد صدفة)
# اخترنا هنا أن يكون الفيلم قد شوهد 50 مرة على الأقل
recommendations = corr_moana[corr_moana['num_of_views'] > 50].sort_values('Correlation', ascending=False)

# نستبعد فيلم "Moana" نفسه من قائمة التوصيات (لأن ارتباطه بنفسه سيكون 1.0 دائماً)
recommendations = recommendations.drop('Moana', errors='ignore')

# نعرض أفضل 5 توصيات
top_5_recommendations = recommendations.head(5)
print(top_5_recommendations)

=== Top 5 Recommendations for users who watched 'Moana' ===

                                       Correlation  num_of_views
program_name                                                    
Storks                                    0.511170          1810
The Jetsons & WWE: Robo-WrestleMania!     0.327318          3812
Inside Out                                0.183280           296
Trolls                                    0.180373         13793
Cloudy With A Chance Of Meatballs         0.174011           426
